In [1]:
#r "C:\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


In [1]:
using BoSSS.Application.SipPoisson;

In [1]:
ExecutionQueues;

In [1]:
var myBatch = (SlurmClient)ExecutionQueues[2];

In [1]:
string WFlowName = "GridCacheError";
WorkflowMgm.Init(WFlowName);
WorkflowMgm.AllJobs;

In [1]:
//string dirname = "DB_SIP";
//string winpath = @"W:\work\scratch\jw52xeqa\"+dirname;
//string linpath = @"/work/scratch/jw52xeqa/"+dirname;

In [1]:
//var tempDB = OpenOrCreateDatabase(winpath);
var tempDB = OpenOrCreateDatabase(@"D:\trash_db");

In [1]:
//MiniBatchProcessor.Server.StartIfNotRunning();

In [1]:
// ===================
// Utility definitions
// ===================

In [1]:
static class Utils {
    // DOF per cell in 3D
    public static int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }    
    
    //Non-equidistant nodes
    public static double[] SinLinSpacing(double l, double r, double a, int n) {
        double[] linnodes = GenericBlas.Linspace(-Math.PI * 0.5, Math.PI * 0.5, n);
        double[] linnodes2 = GenericBlas.Linspace(-1, 1, n);
        double[] nodes = new double[n];

        for (int i = 0; i < n; i++)
            //nodes[i] = linnodes2[i] * (1 - a) + (1.0 - Math.Sin(linnodes[i])) * a;
            nodes[i] = linnodes2[i] * (1 - a) + Math.Sin(linnodes[i])*a;

        for (int i = 0; i < n; i++)
            nodes[i] = nodes[i] * (r - l)*0.5 + l;
        return nodes;
    }
}

In [1]:
// ===============================
// Init grids and save to database
// ===============================

In [1]:
tempDB.Grids;

In [1]:
double Res = Math.Pow(1000000/Utils.Np(2)*64/5,0.333333333);
Res

In [1]:
Math.Pow(109,3)*5

In [1]:
int[] Resolutions_3D = new int[] { 109 };
IGridInfo[] grids = new IGridInfo[Resolutions_3D.Length];
for(int cnt = 0; cnt < Resolutions_3D.Length; cnt++) {
    int Res = Resolutions_3D[cnt];    
    
    double[] xNodes = GenericBlas.Linspace(0, 10, Res*5 + 1);
    double[] yNodes = Utils.SinLinSpacing(-1, +1, 0.6, Res + 1);
    double[] zNodes = Utils.SinLinSpacing(-1, +1, 0.6, Res + 1);
    int J = (xNodes.Length - 1)*(yNodes.Length - 1)*(zNodes.Length - 1);
    
    string GridName = string.Format(WorkflowMgm.CurrentProject + "_J" + J);
    
    grids[cnt]=null;
    foreach(IGridInfo grd in tempDB.Grids){
        bool check = grd.Name.Contains(string.Format("_J"+ J));
        if(check){
          grids[cnt]=grd;
        }
    }
    Console.WriteLine(grids.Length);
  
    //grids[cnt] = tempDB.Grids.SingleOrDefault(grd => grd.Name.Contains(string.Format("_J"+ J));
    if(grids[cnt]==null){
        Console.WriteLine("Creating grid with " + J + " cells.");
        
        GridCommons g;
        g      = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
        g.Name = GridName;
        
        g.EdgeTagNames.Add(1, "Dirichlet");
        g.EdgeTagNames.Add(2, "Neumann");
    
        g.DefineEdgeTags(delegate (double[] X) {
           byte ret;
             if(Math.Abs(X[0] - 0.0) <= 1.0e-6)
               ret = 1;
           else
                ret = 2;
            return ret;
        });
      
        tempDB.SaveGrid(ref g);  
        grids[cnt] = g;
    } else {
        Console.WriteLine("Found Grid: " + grids[cnt]);
        if(grids[cnt].NumberOfCells != J)
            throw new Exception("J mismatch");
        
        if(grids[cnt].SpatialDimension != 3)
            throw new Exception("D mismatch");
    }
    
}

In [1]:
LastError

In [1]:
grids;

In [1]:
// Overwrite Selection
//var ListOfgrids = new List<IGridInfo>();
//ListOfgrids.AddRange(tempDB.Grids.Pick(6));
//IGridInfo[] grids = ListOfgrids.ToArray();
//grids;

In [1]:
// =====================================
// setup control object for a solver run
// =====================================

In [1]:
// - - - - - - - - - - - - - - - - - - -
// Initial Values & Boundary conditions
// - - - - - - - - - - - - - - - - - - -

In [1]:
static class InitialValues {
    public static double RHS(double[] X) {
        return -Math.Sin(X[0]);
    }
    
    public static double DirichletBC(double[] X) {
        return 0.0;
    }
    
    public static double NeumannBC(double[] X) {
       if(Math.Abs(X[1] - 1.0) < 1.0e-8 || Math.Abs(X[1] + 1.0) < 1.0e-8)
           return 0;
       if(X.Length > 2 && (Math.Abs(X[2] - 1.0) < 1.0e-8 || Math.Abs(X[2] + 1.0) < 1.0e-8))
           return 0;

       return Math.Cos(10.0);
   }
}

In [1]:
InitialValues.RHS(new double[]{ 0, 0 });

In [1]:
//myCtrl.NoOfMultigridLevels = 3;

In [1]:
//myCtrl.Run();

In [1]:
// - - - - - - - - - - - - -
// setup of parameter study
// - - - - - - - - - - - - -

In [1]:
List<SipControl> controls = new List<SipControl>();

In [1]:
// polynomial degrees to test
//int[] PolyDegS = new int[] {2, 3, 4, 5, 6};
int[] PolyDegS = new int[] {2,3,5}; // reduced options for dev.

In [1]:
using BoSSS.Solution.AdvancedSolvers;

In [1]:
// Solvers which we want to instrument:
LinearSolverCode[] solver_nameS = new LinearSolverCode[] {
    LinearSolverCode.classic_pardiso, 
    //LinearSolverCode.classic_mumps, 
    LinearSolverCode.exp_Kcycle_schwarz,
    LinearSolverCode.exp_gmres_levelpmg,
}; 

In [1]:
// MaxDof for one claculation (we skip fine grids for higher polynomial orders)
int MaxDof = 5000000;

In [1]:
controls.Clear();
foreach(LinearSolverCode solver_name in solver_nameS) {
foreach(int k in PolyDegS) {   
foreach(IGridInfo grd in grids) {
    var ctrl = new SipControl();
    controls.Add(ctrl);
    
    int Np = Utils.Np(k);
    int J  = grd.NumberOfCells;
    if(J*Np > MaxDof)
        continue;
    
    string caseName = string.Format("J{0}_k{1}_{2}", J, k, solver_name);
    Console.WriteLine("setting up: " + caseName);

    ctrl.AlternateDbPaths = new[]{
        new ValueTuple<string,string>(linpath, ""),
        new ValueTuple<string,string>(winpath, "pcmit32")
    };
    
    ctrl.savetodb = true; //for debug's sake
    ctrl.GridGuid = grd.ID;
    ctrl.FieldOptions.Add("T", new FieldOpts() { Degree = k, SaveToDB = FieldOpts.SaveToDBOpt.TRUE });
    ctrl.FieldOptions.Add("MPIrank", new FieldOpts() { Degree = 0, SaveToDB = FieldOpts.SaveToDBOpt.TRUE });
    ctrl.FieldOptions.Add("Tex", new FieldOpts() { Degree = 0, SaveToDB = FieldOpts.SaveToDBOpt.unspecified});
    ctrl.LinearSolver.SolverCode           = solver_name;
    ctrl.LinearSolver.NoOfMultigridLevels  = int.MaxValue;
    ctrl.LinearSolver.TargetBlockSize      = Math.Min(J*Np-1,10000);
    ctrl.LinearSolver.ConvergenceCriterion = 1e-8;
    
    ctrl.InitialValues.Add("RHS", GetFormulaObject(InitialValues.RHS));
    ctrl.AddBoundaryValue(BoundaryType.Dirichlet.ToString(), "T",
        GetFormulaObject(InitialValues.DirichletBC));
    ctrl.AddBoundaryValue(BoundaryType.Neumann.ToString(), "T",
        GetFormulaObject(InitialValues.NeumannBC));
    ctrl.SessionName = "SIP_"+caseName;
    
    ctrl.GridPartType = BoSSS.Foundation.Grid.GridPartType.METIS;
    //ctrl.GridPartType = BoSSS.Foundation.Grid.GridPartType.directHilbert;
}
}
}

In [1]:
// ==========
// Launch Job
// ==========

In [1]:
// ===old version===
//foreach(var ctrl in controls) {
//    Console.WriteLine(" Submitting: " + ctrl.SessionName);
//    ctrl.RunBatch(myBatch); 
//}

In [1]:
foreach(var ctrl in controls) {
    Console.WriteLine(" Submitting: " + ctrl.SessionName); 
    var aJob = new Job(ctrl.SessionName, typeof(SipPoissonMain));
    aJob.SetControlObject(ctrl);
    ((SlurmClient)myBatch).SlurmAccount = "project01217";  // Jens 
    aJob.NumberOfMPIProcs = 1;
    aJob.ExecutionTime    = "02:00:00";
    //aJob.HHLR_project             = "project01217";
    aJob.MemPerCPU                = "50000";
    aJob.UseComputeNodesExclusive = true;
    aJob.Activate(myBatch);
}

In [1]:
int cnt   = 0;
int iCore = -1;
foreach(var ctrl in controls) {

    if(cnt % JobsWithSameCore == 0)
        iCore++;
         
    Console.WriteLine(" Submitting: " + ctrl.SessionName); 
    var aJob = new Job(ctrl.SessionName, typeof(SipPoissonMain));
    aJob.SetControlObject(ctrl);
    ((SlurmClient)ExecutionQueues[1]).SlurmAccount = "project01217";  // Jens 
    aJob.NumberOfMPIProcs         = core_sweep[iCore];
    aJob.ExecutionTime            = "02:00:00";
    //aJob.HHLR_project             = "project01217";
    aJob.MemPerCPU                = "2500";
    aJob.UseComputeNodesExclusive = true;
    aJob.Activate(myBatch);
    cnt++;
}

In [1]:
LastError

In [1]:
WorkflowMgm.AllJobs.Select(kv => kv.Key + ": \t" + kv.Value.Status);